In [2]:
import numpy as np

import pandas as pd

import pickle

In [20]:
raw_liver=pd.read_csv("Indian Liver Patient Dataset (ILPD).csv")
raw_liver.columns=['Age','Gender','Total_Bilirubin','Direct_Bilirubin','Alkaline_Phosphotase','Alamine_Aminotransferase','Aspartate_Aminotransferase','Total_Protiens','Albumin','Albumin_and_Globulin_Ratio','Target']
col=['Age', 'Total_Bilirubin', 'Direct_Bilirubin', 'Alkaline_Phosphotase',
       'Alamine_Aminotransferase', 'Aspartate_Aminotransferase',
       'Total_Protiens', 'Albumin',
       'Albumin_and_Globulin_Ratio', 'Gender','Target']
data=pd.DataFrame()
for x in col:
    data[x]=raw_liver[x]

In [21]:
def NAN_replace(df,column_name,x):
    #x=mean,median,mode,random,nan_binary,end_distribution
    if x=='mean':
        df[column_name+'_'+x+'_replaced']=df[column_name]
        df[column_name+'_'+x+'_replaced']=df[column_name+'_'+x+'_replaced'].replace(to_replace=np.nan,value=df[column_name].mean(),inplace=False)
    if x=='median':
        df[column_name+'_'+x+'_replaced']=df[column_name]
        df[column_name+'_'+x+'_replaced']=df[column_name+'_'+x+'_replaced'].replace(to_replace=np.nan,value=df[column_name].median(),inplace=False)
    if x=='mode':
        df[column_name+'_'+x+'_replaced']=df[column_name]
        df[column_name+'_'+x+'_replaced']=df[column_name+'_'+x+'_replaced'].replace(to_replace=np.nan,value=df[column_name].mode(),inplace=False)
    if x=='random':
        df[column_name+'_random'] = df[column_name]  
        random_sample = df[column_name].dropna().sample(df[column_name].isnull().sum(), random_state=0)  
        random_sample.index = df[df['Age'].isnull()].index  
        df.loc[df2[column_name].isnull(), column_name+'_random'] = random_sample 
    if x=='nan_binary':
        df[column_name+'_nan_binary']=np.where(d[column_name].isnull(),1,0)
    if x=='end_distribution':
        df[column_name+'_end _distribution']=df[column_name].fillna(df[column_name].mean()+3*df3[column_name].std())
    

In [22]:
NAN_replace(data,'Albumin_and_Globulin_Ratio','median')

In [23]:
data['Gender']=data['Gender'].replace({'Male':1,'Female':0})

C:\Users\FCI\AppData\Local\Temp\ipykernel_5056\2187504411.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Gender']=data['Gender'].replace({'Male':1,'Female':0})


In [24]:
data.drop(columns='Albumin_and_Globulin_Ratio',inplace=True)

In [25]:
x=data.drop(columns='Target')
y=raw_liver['Target']
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy=train_test_split(x,y,test_size=0.25,random_state=47)
print(trainx.shape  ,  trainy.shape)
print(testx.shape  ,   testy.shape)

(436, 11) (436,)
(146, 11) (146,)


In [26]:
from imblearn.combine import SMOTETomek
sm=SMOTETomek()
smot_trainx,smot_trainy=sm.fit_resample(trainx,trainy)
print(trainy.value_counts())
print(smot_trainy.value_counts())

Target
1    312
2    124
Name: count, dtype: int64
Target
2    295
1    295
Name: count, dtype: int64


In [27]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
smot_scl_trainx=ss.fit_transform(smot_trainx)
scl_testx=ss.transform(testx)

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score,confusion_matrix,roc_auc_score

modelcv=GradientBoostingClassifier(subsample= 1, n_estimators= 300, min_samples_split= 10,
                                   min_samples_leaf= 1, max_features= 'sqrt', max_depth= 5, learning_rate= 1,random_state=47)
modelcv.fit(smot_scl_trainx,smot_trainy)
ypred_dt=modelcv.predict(scl_testx)

print('f1_score        ',f1_score(ypred_dt,testy))
print('accuracy_score  ',accuracy_score(ypred_dt,testy))
print('precision_score ',precision_score(ypred_dt,testy))
print('recall_score    ',recall_score(ypred_dt,testy))
print('classification_report')
print(classification_report(ypred_dt,testy))
print('confusion_matrix')
print(confusion_matrix(ypred_dt,testy))

f1_score         0.7439613526570048
accuracy_score   0.636986301369863
precision_score  0.7475728155339806
recall_score     0.7403846153846154
classification_report
              precision    recall  f1-score   support

           1       0.75      0.74      0.74       104
           2       0.37      0.38      0.38        42

    accuracy                           0.64       146
   macro avg       0.56      0.56      0.56       146
weighted avg       0.64      0.64      0.64       146

confusion_matrix
[[77 27]
 [26 16]]


In [13]:
pickle.dump(ss, open('StandardScaler.pkl','wb'))
pickle.dump(modelcv, open('model.pkl','wb'))

In [14]:

model = pickle.load(open('model.pkl','rb'))
print(model.predict([[62,1.0,0.4,182,14,20,6.8,3.4,1,0.74]]))

[1]


In [13]:
x=trainx.columns


Index(['Age', 'Total_Bilirubin', 'Direct_Bilirubin', 'Alkaline_Phosphotase',
       'Alamine_Aminotransferase', 'Aspartate_Aminotransferase',
       'Total_Protiens', 'Albumin', 'Gender',
       'Albumin_and_Globulin_Ratio_median_replaced'],
      dtype='object')

In [18]:
x=['Target','Age', 'Total_Bilirubin', 'Direct_Bilirubin', 'Alkaline_Phosphotase',
       'Alamine_Aminotransferase', 'Aspartate_Aminotransferase',
       'Total_Protiens', 'Albumin', 'Gender',
       'Albumin_and_Globulin_Ratio_median_replaced']

In [33]:
smot_trainx


,Age,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Gender,Target,Albumin_and_Globulin_Ratio_median_replaced
0,22,1.100000,0.300000,138,14,21,7.000000,3.800000,0,2,1.100000
1,46,20.000000,10.000000,254,140,540,5.400000,3.000000,1,1,1.200000
2,33,1.500000,7.000000,505,205,140,7.500000,3.900000,1,1,1.000000
3,25,0.900000,0.300000,159,24,25,6.900000,4.400000,0,2,1.700000
4,26,7.100000,3.300000,258,80,113,6.200000,2.900000,1,1,0.800000
...,...,...,...,...,...,...,...,...,...,...,...
585,62,0.935032,0.305839,309,60,60,6.941613,3.376645,1,2,0.905839
586,19,0.604128,0.152064,208,20,22,7.204128,4.291745,0,2,1.439681
587,25,0.833053,0.277684,159,21,22,6.922316,4.199158,0,2,1.543789
588,47,0.800000,0.200000,224,24,22,5.706910,2.635176,0,2,0.852688


In [38]:
trainy

529    2
6      1
508    1
102    2
260    1
      ..
59     1
23     2
264    2
327    2
135    1
Name: Target, Length: 436, dtype: int64